## Uninominal One-Stage Voting

In [1]:
# uninominal one-stage voting (plurality voting)
def plurality(n_candidates, n_voters, votes, candidates):
    
    count = [0] * n_candidates
    winner = 'unknown'
    
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
        
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
        
    else:
        for v in votes:
            current_order = v[1]
            for i in range(n_candidates):
                if current_order.find(candidates[i]) == 0:
                    count[i] += v[0]
        for i in range(n_candidates):
            print("count(",candidates[i], "): ", count[i])
        
        winner = candidates[count.index(max(count))]
        
    return count, winner

## Uninominal Two-Stage Voting

In [2]:
# uninominal two-stage voting system
from heapq import nlargest
import copy

def two_stage(n_candidates, n_voters, votes, candidates):
    print("1st stage:")
    # first round
    counts, winner = plurality(n_candidates, n_voters, votes, candidates)
    # check if a candidate has the absolute majority
    if max(counts) > n_voters/2:
        print("winner:", winner, "has the absolute majority")
        print("----------")
        return counts, winner
    else:
        two_cand = nlargest(2, enumerate(counts), key=lambda x: x[1])
        print("candidates for 2nd round:", candidates[two_cand[0][0]], candidates[two_cand[1][0]])
        print("----------")
        
        votes_copy = copy.deepcopy(votes)
        for v in votes_copy:
            v[1] = ''.join(c for c in v[1] if c == candidates[two_cand[0][0]] or c == candidates[two_cand[1][0]])
        
        candidates = [candidates[two_cand[0][0]], candidates[two_cand[1][0]]]
        print("2nd stage:")
        counts, winner = plurality(2, n_voters, votes_copy, candidates)
        print("winner:", winner)
        print("----------")

## Sequential Voting (with agenda)

In [3]:
# sequential voting
def sequential_voting(n_candidates, n_voters, votes, agenda):

    winner = 'unknown'
    
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
        
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
        
    else:
        winner = agenda[0]
        for i in range(n_candidates-1):
            count = [0] * 2
            challenger = agenda[i+1]
            for v in votes:
                order = [s for s in v[1]]
                if order.index(winner) < order.index(challenger):
                    count[0] += v[0]
                else:
                    count[1] += v[0]
            print(winner, "(", count[0], ") vs", challenger, "(", count[1], ")", end='')
            if count[0] < count[1]:
                winner = challenger
            print(" ==>", winner)
    print("winner:", winner)
    return winner

## Condorcet Winner

In [4]:
# condorcet winner
from itertools import combinations 
def condorcet_winner(n_candidates, n_voters, votes):
    
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
        
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
        
    else:
        # can = candidates[:n_candidates]
        #combinations_can = sum([list(map(list, combinations(can, 2)))], [])
        combinations_can = sum([list(map(list, combinations(candidates, 2)))], [])
        
        for pair in combinations_can:
            count = [0] * 2
            winner = pair[0]
            challenger = pair[1]
            for v in votes:
                order = [s for s in v[1]]
                if order.index(winner) < order.index(challenger):
                    count[0] += v[0]
                else:
                    count[1] += v[0]
            print(winner, "vs", challenger, end='')
            if count[0] > count[1]:
                print("   ===>", winner, "(", count[0], ") >", challenger, "(", count[1], ")")
            if count[0] < count[1]:
                print("   ===>", challenger, "(", count[1], ") >", winner, "(", count[0], ")")

## Borda scoring

In [5]:
# Borda method
def borda(n_candidates, n_voters, votes, candidates):
    
    count = [0] * n_candidates
    
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
        
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
        
    else:
        for v in votes:
            current_order = v[1]
            for i in range(n_candidates):
                rank_i = current_order.find(candidates[i])
                count[i] += (n_candidates - 1 - rank_i) * v[0]
                
        for i in range(n_candidates):
            print("score(",candidates[i], "): ", count[i])
        
    return count

## Borda elimination method

In [6]:
# Borda elimination method
import heapq
import copy

def borda_elimination(n_candidates, n_voters, votes, candidates):
    
    votes_copy = copy.deepcopy(votes)
    candidates_copy = copy.deepcopy(candidates)
    
    while(len(candidates_copy) > 1):
        scores = borda(n_candidates, n_voters, votes_copy, candidates_copy)
        
        eliminated_cand = heapq.nsmallest(1, enumerate(scores), key=lambda x: x[1])
        # print(eliminated_cand)
        print("borda eliminated candidate:", candidates_copy[eliminated_cand[0][0]], ", its score:", eliminated_cand[0][1])
        
        for v in votes_copy:
            v[1] = ''.join(c for c in v[1] if c != candidates_copy[eliminated_cand[0][0]])
        candidates_copy.remove(candidates_copy[eliminated_cand[0][0]])
        n_candidates = n_candidates - 1
        
    print("winner:", candidates_copy[0])
    votes = copy.deepcopy(votes_copy)
    candidates = copy.deepcopy(candidates_copy)

# Examples

In [7]:
# one-stage && two-stage
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 21
votes = [
    [10, "abc"],
    [6, "bca"],
    [5, "cba"]
]

counts, winner = plurality(n_candidates, n_voters, votes, candidates)
print("counts:", counts, "==> winner:", winner)
print("---------------")
two_stage(n_candidates, n_voters, votes, candidates)

count( a ):  10
count( b ):  6
count( c ):  5
counts: [10, 6, 5] ==> winner: a
---------------
1st stage:
count( a ):  10
count( b ):  6
count( c ):  5
candidates for 2nd round: a b
----------
2nd stage:
count( a ):  10
count( b ):  11
winner: b
----------


In [8]:
# sequential voting
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 3
votes = [
    [1, "abc"],
    [1, "bca"],
    [1, "cab"]
]
agenda = ['a', 'b', 'c']
print("sequantial voting")
winner = sequential_voting(n_candidates, n_voters, votes, agenda)
print("---------------")
agenda = ['b', 'c', 'a']
print("sequantial voting")
winner = sequential_voting(n_candidates, n_voters, votes, agenda)
print("---------------")
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 3
votes = [
    [1, "badc"],
    [1, "cbad"],
    [1, "adcb"]
]
agenda = ['a', 'b', 'c', 'd']
print("sequantial voting")
winner = sequential_voting(n_candidates, n_voters, votes, agenda)

sequantial voting
a ( 2 ) vs b ( 1 ) ==> a
a ( 1 ) vs c ( 2 ) ==> c
winner: c
---------------
sequantial voting
b ( 2 ) vs c ( 1 ) ==> b
b ( 1 ) vs a ( 2 ) ==> a
winner: a
---------------
sequantial voting
a ( 1 ) vs b ( 2 ) ==> b
b ( 1 ) vs c ( 2 ) ==> c
c ( 1 ) vs d ( 2 ) ==> d
winner: d


In [9]:
# condorcet winner
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 3
votes = [
    [1, "abc"],
    [1, "acb"],
    [1, "cba"]
]
print("condorcet winner")
condorcet_winner(n_candidates, n_voters, votes)
print("---------------")

candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 21
votes = [
    [10, "abc"],
    [6, "bca"],
    [5, "cba"]
]
counts, winner = plurality(n_candidates, n_voters, votes, candidates)
print("counts:", counts, "==> winner:", winner)
print("----------")
print("condorcet winner")
condorcet_winner(n_candidates, n_voters, votes)
print("---------------")

candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 21
votes = [
    [10, "bacd"],
    [6, "cadb"],
    [5, "adbc"]
]
two_stage(n_candidates, n_voters, votes, candidates)
print("----------")
print("condorcet winner")
condorcet_winner(n_candidates, n_voters, votes)
print("---------------")

candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 3
votes = [
    [2, "bacd"],
    [1, "acdb"]
]
borda(n_candidates, n_voters, votes, candidates)
print("----------")
print("condorcet winner")
condorcet_winner(n_candidates, n_voters, votes)
print("---------------")

candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 7
votes = [
    [3, "cab"],
    [2, "abc"],
    [1, "acb"],
    [1, "bca"],
]
borda(n_candidates, n_voters, votes, candidates)
print("----------")
print("condorcet winner")
condorcet_winner(n_candidates, n_voters, votes)

condorcet winner
a vs b   ===> a ( 2 ) > b ( 1 )
a vs c   ===> a ( 2 ) > c ( 1 )
b vs c   ===> c ( 2 ) > b ( 1 )
---------------
count( a ):  10
count( b ):  6
count( c ):  5
counts: [10, 6, 5] ==> winner: a
----------
condorcet winner
a vs b   ===> b ( 11 ) > a ( 10 )
a vs c   ===> c ( 11 ) > a ( 10 )
b vs c   ===> b ( 16 ) > c ( 5 )
---------------
1st stage:
count( a ):  5
count( b ):  10
count( c ):  6
count( d ):  0
candidates for 2nd round: b c
----------
2nd stage:
count( b ):  15
count( c ):  6
winner: b
----------
----------
condorcet winner
a vs b   ===> a ( 11 ) > b ( 10 )
a vs c   ===> a ( 15 ) > c ( 6 )
a vs d   ===> a ( 21 ) > d ( 0 )
b vs c   ===> b ( 15 ) > c ( 6 )
b vs d   ===> d ( 11 ) > b ( 10 )
c vs d   ===> c ( 16 ) > d ( 5 )
---------------
score( a ):  7
score( b ):  6
score( c ):  4
score( d ):  1
----------
condorcet winner
a vs b   ===> b ( 2 ) > a ( 1 )
a vs c   ===> a ( 3 ) > c ( 0 )
a vs d   ===> a ( 3 ) > d ( 0 )
b vs c   ===> b ( 2 ) > c ( 1 )
b vs d   ==

In [10]:
# borda elimination method
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 7
votes = [
    [3, "cab"],
    [2, "abc"],
    [1, "acb"],
    [1, "bca"],
]
print("borda elimination method")
borda_elimination(n_candidates, n_voters, votes, candidates)
print("---------------")

candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 3
votes = [
    [2, "bacd"],
    [1, "acdb"]
]
print("borda elimination method")
borda_elimination(n_candidates, n_voters, votes, candidates)
print("---------------")

candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 27
votes = [
    [6, "abc"],
    [4, "bac"],
    [6, "bca"],
    [2, "cba"],
    [6, "cab"],
    [3, "acb"]
]
print("borda elimination method")
borda_elimination(n_candidates, n_voters, votes, candidates)
print("----------")
print("\n")
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 27
votes = [
    [9, "abc"],
    [1, "bac"],
    [6, "bca"],
    [8, "cab"],
    [3, "acb"]
]
print("borda elimination method")
borda_elimination(n_candidates, n_voters, votes, candidates)

borda elimination method
score( a ):  9
score( b ):  4
score( c ):  8
borda eliminated candidate: b , its score: 4
score( a ):  3
score( c ):  4
borda eliminated candidate: a , its score: 3
winner: c
---------------
borda elimination method
score( a ):  7
score( b ):  6
score( c ):  4
score( d ):  1
borda eliminated candidate: d , its score: 1
score( a ):  4
score( b ):  4
score( c ):  1
borda eliminated candidate: c , its score: 1
score( a ):  1
score( b ):  2
borda eliminated candidate: a , its score: 1
winner: b
---------------
borda elimination method
score( a ):  28
score( b ):  28
score( c ):  25
borda eliminated candidate: c , its score: 25
score( a ):  15
score( b ):  12
borda eliminated candidate: b , its score: 12
winner: a
----------


borda elimination method
score( a ):  33
score( b ):  23
score( c ):  25
borda eliminated candidate: b , its score: 23
score( a ):  13
score( c ):  14
borda eliminated candidate: a , its score: 13
winner: c


## Kramer-Simpson, Copeland

In [11]:
# kramer-simpson 
from itertools import combinations
import numpy as np

def kramer_simpson(n_candidates, n_voters, votes, candidates):
    
    mat = np.full((n_candidates, n_candidates), np.inf)
    
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
        
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
        
    else:
        combinations_cand_index = sum([list(map(list, combinations(range(n_candidates), 2)))], [])
        
        for pair in combinations_cand_index:
            winner = pair[0]
            challenger = pair[1]
            mat[winner][challenger] = 0
            mat[challenger][winner] = 0
            
            for v in votes:
                order = [s for s in v[1]]
                if order.index(candidates[winner]) < order.index(candidates[challenger]):
                    mat[winner][challenger] += v[0]
                else:
                    mat[challenger][winner] += v[0]
    return mat

# Copeland
def copeland(n_candidates, n_voters, votes, candidates):
    mat_co = np.zeros((n_candidates, n_candidates))
    mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
    
    combinations_cand_index = sum([list(map(list, combinations(range(n_candidates), 2)))], [])
    for pair in combinations_cand_index:
        if mat_ks[pair[0]][pair[1]] > mat_ks[pair[1]][pair[0]]:
            mat_co[pair[0]][pair[1]] = 1
            mat_co[pair[1]][pair[0]] = -1
        else:
            mat_co[pair[0]][pair[1]] = -1
            mat_co[pair[1]][pair[0]] = 1
    return mat_co

In [12]:
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 3

votes = [
    [1, "abc"],
    [1, "acb"],
    [1, "cba"],
]

mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
print(mat_ks)
print("KramerSimpson-min", mat_ks.min(axis=1))
print("----------")
mat_co = copeland(n_candidates, n_voters, votes, candidates)
print(mat_co)
print("Copeland-sum", mat_co.sum(axis=1))
print("----------")

[[inf  2.  2.]
 [ 1. inf  1.]
 [ 1.  2. inf]]
KramerSimpson-min [2. 1. 1.]
----------
[[ 0.  1.  1.]
 [-1.  0. -1.]
 [-1.  1.  0.]]
Copeland-sum [ 2. -2.  0.]
----------


In [13]:
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 12

votes = [
    [5, "abcd"],
    [4, "bcda"],
    [3, "dcab"],
]

mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
print(mat_ks)
print("KramerSimpson-min", mat_ks.min(axis=1))
print("----------")
mat_co = copeland(n_candidates, n_voters, votes, candidates)
print(mat_co)
print("Copeland-sum", mat_co.sum(axis=1))
print("----------")

[[inf  8.  5.  5.]
 [ 4. inf  9.  9.]
 [ 7.  3. inf  9.]
 [ 7.  3.  3. inf]]
KramerSimpson-min [5. 4. 3. 3.]
----------
[[ 0.  1. -1. -1.]
 [-1.  0.  1.  1.]
 [ 1. -1.  0.  1.]
 [ 1. -1. -1.  0.]]
Copeland-sum [-1.  1.  1. -1.]
----------


In [14]:
candidates = ['a', 'b', 'c', 'd', 'e']
n_candidates = 5
n_voters = 9

votes = [
    [1, "abcde"],
    [4, "cdbea"],
    [1, "eadbc"],
    [3, "eabdc"],
]

mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
print(mat_ks)
print("KramerSimpson-min", mat_ks.min(axis=1))
print("----------")
mat_co = copeland(n_candidates, n_voters, votes, candidates)
print(mat_co)
print("Copeland-sum", mat_co.sum(axis=1))
print("----------")
borda(n_candidates, n_voters, votes, candidates)

[[inf  5.  5.  5.  1.]
 [ 4. inf  5.  4.  5.]
 [ 4.  4. inf  5.  5.]
 [ 4.  5.  4. inf  5.]
 [ 8.  4.  4.  4. inf]]
KramerSimpson-min [1. 4. 4. 4. 4.]
----------
[[ 0.  1.  1.  1. -1.]
 [-1.  0.  1. -1.  1.]
 [-1. -1.  0.  1.  1.]
 [-1.  1. -1.  0.  1.]
 [ 1. -1. -1. -1.  0.]]
Copeland-sum [ 2.  0.  0.  0. -2.]
----------
score( a ):  16
score( b ):  18
score( c ):  18
score( d ):  18
score( e ):  20


[16, 18, 18, 18, 20]

## Alternative voting (Instant run-off voting)

In [15]:
# alternative voting
import heapq
import copy
def alternative_voting(n_candidates, n_voters, votes, candidates):
    
    votes_copy = copy.deepcopy(votes)
    candidates_copy = copy.deepcopy(candidates)
    
    condition = True
    while condition:
        scores, winner = plurality(n_candidates, n_voters, votes_copy, candidates_copy)
        if max(scores) > n_voters/2:
            condition = False
        else:
            eliminated_cand = heapq.nsmallest(1, enumerate(scores), key=lambda x: x[1])
            print("eliminated candidate:", candidates_copy[eliminated_cand[0][0]], ", its score:", eliminated_cand[0][1])
            
            for v in votes_copy:
                v[1] = ''.join(c for c in v[1] if c != candidates_copy[eliminated_cand[0][0]])
            candidates_copy.remove(candidates_copy[eliminated_cand[0][0]])
            n_candidates = n_candidates - 1
    
    winner = candidates_copy[scores.index(max(scores))]
    print("winner:", winner)

In [16]:
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 100

votes = [
    [28, "abcd"],
    [14, "acdb"],
    [15, "bcad"],
    [17, "cabd"],
    [26, "dbca"],
]

alternative_voting(n_candidates, n_voters, votes, candidates)

count( a ):  42
count( b ):  15
count( c ):  17
count( d ):  26
eliminated candidate: b , its score: 15
count( a ):  42
count( c ):  32
count( d ):  26
eliminated candidate: d , its score: 26
count( a ):  42
count( c ):  58
winner: c


In [17]:
candidates = ['a', 'b', 'c']
n_candidates = 3
n_voters = 21

votes = [
    [10, "bac"],
    [6, "cab"],
    [5, "abc"],
]

alternative_voting(n_candidates, n_voters, votes, candidates)
print("\n")
condorcet_winner(n_candidates, n_voters, votes)

count( a ):  5
count( b ):  10
count( c ):  6
eliminated candidate: a , its score: 5
count( b ):  15
count( c ):  6
winner: b


a vs b   ===> a ( 11 ) > b ( 10 )
a vs c   ===> a ( 15 ) > c ( 6 )
b vs c   ===> b ( 15 ) > c ( 6 )


## Coombs

In [18]:
import heapq
import copy
def coombs(n_candidates, n_voters, votes, candidates):
    votes_copy = copy.deepcopy(votes)
    candidates_copy = copy.deepcopy(candidates)
    condition = True
    while condition:
        count_first, count_last = first_last_count(n_candidates, n_voters, votes_copy, candidates_copy)
        if max(count_first) > n_voters/2:
            print("absolute majority", end=" | ")
            condition = False
        else:
            print("no absolute majority", end=" | ")
            eliminated_cand = heapq.nlargest(1, enumerate(count_last), key=lambda x: x[1])
            print("eliminated candidate:", candidates_copy[eliminated_cand[0][0]], ",", eliminated_cand[0][1])
            
            for v in votes_copy:
                v[1] = ''.join(c for c in v[1] if c != candidates_copy[eliminated_cand[0][0]])
            candidates_copy.remove(candidates_copy[eliminated_cand[0][0]])
            n_candidates = n_candidates - 1
    winner = candidates_copy[count_first.index(max(count_first))]
    print("winner:", winner)
    votes = copy.deepcopy(votes_copy)
    candidates = copy.deepcopy(candidates_copy)

def first_last_count(n_candidates, n_voters, votes, candidates):
    count_first = [0] * n_candidates
    count_last = [0] * n_candidates
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
    else:
        for v in votes:
            current_order = v[1]
            for i in range(n_candidates):
                if current_order.find(candidates[i]) == 0:
                    count_first[i] += v[0]
                if current_order.find(candidates[i]) == n_candidates-1:
                    count_last[i] += v[0]
        for i in range(n_candidates):
            print("score(",candidates[i], "): ", count_first[i], count_last[i])
    return count_first, count_last

In [19]:
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 100

votes = [
    [28, "abcd"],
    [14, "acdb"],
    [15, "bcad"],
    [17, "cabd"],
    [26, "dbca"],
]

coombs(n_candidates, n_voters, votes, candidates)

score( a ):  42 26
score( b ):  15 14
score( c ):  17 0
score( d ):  26 60
no absolute majority | eliminated candidate: d , 60
score( a ):  42 41
score( b ):  41 31
score( c ):  17 28
no absolute majority | eliminated candidate: a , 41
score( b ):  69 31
score( c ):  31 69
absolute majority | winner: b


## Hare

In [20]:
# Hare
import heapq
import copy
import math

def count_first(n_candidates, n_voters, votes, candidates):
    counts = [0] * n_candidates
    if n_voters != sum(v[0] for v in votes):
        print("Error! n_voters")
    elif n_candidates != len(votes[0][1]): 
        print("Error! n_candidates")
    else:
        for v in votes:
            current_order = v[1]
            for i in range(n_candidates):
                if current_order.find(candidates[i]) == 0:
                    counts[i] += v[0]
        for i in range(n_candidates):
            print("count(",candidates[i], "): ", counts[i])
    return counts

def hare(n_candidates, n_voters, n_seats, votes, candidates):
    votes_copy = copy.deepcopy(votes)
    candidates_copy = copy.deepcopy(candidates)
    
    quota = math.floor(n_voters/(n_seats+1))+1
    n_elected = 0
    
    while n_elected < n_seats:
        counts = count_first(n_candidates, n_voters, votes_copy, candidates_copy)
        cand_among_quota = [idx for idx,value in enumerate(counts) if value >= quota]
        
        if len(cand_among_quota) == 0:
            eliminated_cand = heapq.nsmallest(1, enumerate(counts), key=lambda x: x[1])
            print("eliminated candidate:", candidates_copy[eliminated_cand[0][0]])
            print("----------")
            
            for v in votes_copy:
                v[1] = ''.join(c for c in v[1] if c != candidates_copy[eliminated_cand[0][0]])
            candidates_copy.remove(candidates_copy[eliminated_cand[0][0]])
            n_candidates = n_candidates - 1
            
        else:
            cand_idx_among_quota = [index for index,value in enumerate(counts) if value >= quota]
            counts_among_quota = [counts[i] for i in cand_idx_among_quota]
            #print(cand_idx_among_quota, counts_among_quota)
            
            for idx in cand_idx_among_quota:
                n_elected += 1
                print("candidates elected:", candidates_copy[idx])
                print("----------")
                
                if counts[idx] > quota:
                    surplus_votes = counts[idx] - quota
                    for v in votes_copy:
                        current_order = v[1]
                        if current_order.find(candidates_copy[idx]) == 0:
                            v[0] = (v[0]/counts[idx])*surplus_votes
            
            candidates_tmp = copy.deepcopy(candidates_copy)
            for idx in cand_idx_among_quota:
                for v in votes_copy:
                    can = candidates_tmp[idx]
                    v[1] = ''.join(c for c in v[1] if c != can)
                candidates_copy.remove(can)
                n_candidates -= 1
                n_voters -= quota

In [21]:
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 100
votes = [
    [28, "abcd"],
    [14, "acdb"],
    [15, "bcad"],
    [17, "cabd"],
    [26, "dbca"],
]
n_seats = 2
hare(n_candidates, n_voters, n_seats, votes, candidates)

count( a ):  42
count( b ):  15
count( c ):  17
count( d ):  26
candidates elected: a
----------
count( b ):  20.333333333333332
count( c ):  19.666666666666668
count( d ):  26
eliminated candidate: c
----------
count( b ):  37.33333333333333
count( d ):  28.666666666666668
candidates elected: b
----------


## Largest reminders method

In [22]:
# Largest reminders
import math
import numpy
def largest_reminder(n_list_of_candidates, n_seats, n_voters, list_of_candidates, votes_plurinominal):
    
    q = n_voters/n_seats
    
    vq = [math.floor(v/q) for v in votes_plurinominal]
    
    vr = [votes_plurinominal[idx]-(vq[idx]*q) for idx in range(len(votes_plurinominal))]
    
    n_seats_remain = n_seats - sum(vq)
    
    allocation_remain = heapq.nlargest(n_seats_remain, enumerate(vr), key=lambda x: x[1])
    
    for pair in allocation_remain:
        idx = pair[0]
        vq[idx] += 1
    
    for idx in range(n_list_of_candidates):
        print(list_of_candidates[idx], ":", vq[idx])

In [28]:
n_seats = 5
n_list_of_candidates = 4
n_voters = 100
list_of_candidates = ['A', 'B', 'C', 'D']
votes_plurinominal  = [23, 29, 32, 16]
largest_reminder(n_list_of_candidates, n_seats, n_voters, list_of_candidates, votes_plurinominal)

A : 1
B : 1
C : 2
D : 1


In [31]:
candidates = ['a', 'b', 'c', 'd']
n_candidates = 4
n_voters = 100
votes = [
    [32, "cabd"],
    [23, "adcb"],
    [16, "dcab"],
    [29, "badc"],
]

print("one-stage:")
counts, winner = plurality(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
two_stage(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
#agenda = ['a', 'b', 'c']
#winner = sequential_voting(n_candidates, n_voters, votes, agenda)
#print("--------------------")
print("condorcet winner:")
condorcet_winner(n_candidates, n_voters, votes)
print("----------------------------------------")
print("borda:")
borda(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("borda elimination:")
borda_elimination(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("kramer-simpson:")
mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
print(mat_ks)
print("KramerSimpson-min", mat_ks.min(axis=1))
print("----------------------------------------")
print("Copeland:")
mat_co = copeland(n_candidates, n_voters, votes, candidates)
print(mat_co)
print("Copeland-sum", mat_co.sum(axis=1))
print("----------------------------------------")
print("Alternative voting:")
alternative_voting(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("Coombs:")
coombs(n_candidates, n_voters, votes, candidates)
print("Hare:")
n_seats = 2
hare(n_candidates, n_voters, n_seats, votes, candidates)

one-stage:
count( a ):  23
count( b ):  29
count( c ):  32
count( d ):  16
----------------------------------------
1st stage:
count( a ):  23
count( b ):  29
count( c ):  32
count( d ):  16
candidates for 2nd round: c b
----------
2nd stage:
count( c ):  71
count( b ):  29
winner: c
----------
----------------------------------------
condorcet winner:
a vs b   ===> a ( 71 ) > b ( 29 )
a vs c   ===> a ( 52 ) > c ( 48 )
a vs d   ===> a ( 84 ) > d ( 16 )
b vs c   ===> c ( 71 ) > b ( 29 )
b vs d   ===> b ( 61 ) > d ( 39 )
c vs d   ===> d ( 68 ) > c ( 32 )
----------------------------------------
borda:
score( a ):  207
score( b ):  119
score( c ):  151
score( d ):  123
----------------------------------------
borda elimination:
score( a ):  207
score( b ):  119
score( c ):  151
score( d ):  123
borda eliminated candidate: b , its score: 119
score( a ):  136
score( c ):  80
score( d ):  84
borda eliminated candidate: c , its score: 80
score( a ):  84
score( d ):  16
borda eliminated candid

In [30]:
candidates = ['a', 'b', 'c', 'd', 'e']
n_candidates = 5
n_voters = 100
votes = [
    [30, "abcde"],
    [21, "cdaeb"],
    [16, "bcaed"],
    [33, "edbca"],
]

print("one-stage:")
counts, winner = plurality(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
two_stage(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
#agenda = ['a', 'b', 'c']
#winner = sequential_voting(n_candidates, n_voters, votes, agenda)
#print("--------------------")
print("condorcet winner:")
condorcet_winner(n_candidates, n_voters, votes)
print("----------------------------------------")
print("borda:")
borda(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("borda elimination:")
borda_elimination(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("kramer-simpson:")
mat_ks = kramer_simpson(n_candidates, n_voters, votes, candidates)
print(mat_ks)
print("KramerSimpson-min", mat_ks.min(axis=1))
print("----------------------------------------")
print("Copeland:")
mat_co = copeland(n_candidates, n_voters, votes, candidates)
print(mat_co)
print("Copeland-sum", mat_co.sum(axis=1))
print("----------------------------------------")
print("Alternative voting:")
alternative_voting(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("Coombs:")
coombs(n_candidates, n_voters, votes, candidates)
print("----------------------------------------")
print("Hare:")
n_seats = 2
hare(n_candidates, n_voters, n_seats, votes, candidates)

one-stage:
count( a ):  30
count( b ):  16
count( c ):  21
count( d ):  0
count( e ):  33
----------------------------------------
1st stage:
count( a ):  30
count( b ):  16
count( c ):  21
count( d ):  0
count( e ):  33
candidates for 2nd round: e a
----------
2nd stage:
count( e ):  33
count( a ):  67
winner: a
----------
----------------------------------------
condorcet winner:
a vs b   ===> a ( 51 ) > b ( 49 )
a vs c   ===> c ( 70 ) > a ( 30 )
a vs d   ===> d ( 54 ) > a ( 46 )
a vs e   ===> a ( 67 ) > e ( 33 )
b vs c   ===> b ( 79 ) > c ( 21 )
b vs d   ===> d ( 54 ) > b ( 46 )
b vs e   ===> e ( 54 ) > b ( 46 )
c vs d   ===> c ( 67 ) > d ( 33 )
c vs e   ===> c ( 67 ) > e ( 33 )
d vs e   ===> d ( 51 ) > e ( 49 )
----------------------------------------
borda:
score( a ):  194
score( b ):  220
score( c ):  225
score( d ):  192
score( e ):  169
----------------------------------------
borda elimination:
score( a ):  194
score( b ):  220
score( c ):  225
score( d ):  192
score( e ):  1